In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
import pandas
%matplotlib notebook
import general
import discretionary_aua
import vantage_aua
from data_access.data_accessing import pull_data_for_model, read_assumptions_from_google_sheet
import revenue
import costs
import consolidated
import discf
import combined
from ipywidgets import widgets, Output, interact, IntSlider
from IPython.display import display, clear_output, DisplayHandle
import stats
pandas.set_option('display.float_format', '{0:,.4f}'.format)               
idx = pandas.IndexSlice
cached_displays = {}

In [ ]:
print('Pulling data...')

data_dic = pull_data_for_model()

input_dic = read_assumptions_from_google_sheet()

print('Data pulled successfully.')

In [ ]:
assumption_button = widgets.Button(description='Update assumptions',layout=widgets.Layout(width='20%',height='40px'))
assumption_button.style.button_color = 'lightgreen'
def update_assumption(b):
    global input_dic
    input_dic = read_assumptions_from_google_sheet()
    print ('Assumptions updated successfully...')

assumption_button.on_click(update_assumption)
display(assumption_button)

In [ ]:
def freeze_header(df, num_rows=25, num_columns=3, step_rows=1,
                  step_columns=1):
    """
    Freeze the headers (column and index names) of a Pandas DataFrame. A widget
    enables to slide through the rows and columns.

    Parameters
    ----------
    df : Pandas DataFrame
        DataFrame to display
    num_rows : int, optional
        Number of rows to display
    num_columns : int, optional
        Number of columns to display
    step_rows : int, optional
        Step in the rows
    step_columns : int, optional
        Step in the columns

    Returns
    -------
    Displays the DataFrame with the widget
    """
    @interact(last_row=IntSlider(min=min(num_rows, df.shape[0]),
                                 max=df.shape[0],
                                 step=step_rows,
                                 description='rows',
                                 readout=False,
                                 disabled=False,
                                 continuous_update=True,
                                 orientation='horizontal',
                                 slider_color='purple'),
              last_column=IntSlider(min=min(num_columns, df.shape[1]),
                                    max=df.shape[1],
                                    step=step_columns,
                                    description='columns',
                                    readout=False,
                                    disabled=False,
                                    continuous_update=True,
                                    orientation='horizontal',
                                    slider_color='purple'))
    def _freeze_header(last_row, last_column):
        display(df.iloc[max(0, last_row-num_rows):last_row,
                        max(0, last_column-num_columns):last_column])

In [ ]:
def display_with_out(f, out_obj, *args, **kwargs):
    with out_obj:
        index_slice = kwargs.get('slice', None) # list of indices
        if index_slice:
            result = f(*args).loc[index_slice]
        else:
            result = f(*args, **kwargs)
        if result is not None:
            if isinstance(result, pandas.DataFrame):
                freeze_header(result)
            else:
                display(result)
            return result

In [ ]:
out = Output()

revenue_types = ['Semi-annual revenue','Annual revenue - FY','Annual revenue - CY','Consolidated semi-annual revenue','Consolidated annual revenue - FY','Consolidated annual revenue - CY', 'clear outputs']
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch',border='solid',width='30%')
items = {}
for w in revenue_types:
    if w == 'clear outputs':
        items[w] = widgets.Button(description=w, layout=widgets.Layout(width='100%', height='40px'), button_style='warning', icon='check')
    else:
        items[w] = widgets.Button(description=w,layout=widgets.Layout(width='100%', height='40px'))
        items[w].style.button_color = 'lightgreen'
left_box = widgets.VBox([list(items.values())[0],list(items.values())[1],list(items.values())[2]],layout=box_layout)
right_box = widgets.VBox([list(items.values())[3],list(items.values())[4],list(items.values())[5], list(items.values())[6]],layout=box_layout)
items['Semi-annual revenue'].on_click(lambda x: display_with_out(revenue.semi_revenue,out,data_dic,input_dic))
items['Annual revenue - FY'].on_click(lambda x: display_with_out(revenue.annual_revenue,out,data_dic,input_dic))
items['Annual revenue - CY'].on_click(lambda x: display_with_out(revenue.annual_revenue,out,data_dic,input_dic,cal_year=True))
items['Consolidated semi-annual revenue'].on_click(lambda x: display_with_out(consolidated.revenue_analysis,out,data_dic,input_dic))
items['Consolidated annual revenue - FY'].on_click(lambda x: display_with_out(consolidated.annual_revenue_analysis,out,data_dic,input_dic))
items['Consolidated annual revenue - CY'].on_click(lambda x: display_with_out(consolidated.annual_revenue_analysis,out,data_dic,input_dic,cal_year=True))
items['clear outputs'].on_click(lambda x: display_with_out(clear_output,out))
widgets.HBox([left_box,right_box])




In [ ]:
display(out)

In [ ]:
out2 = Output()

costs_types = ['Semi-annual costs','Annual costs - FY','Annual costs - CY','Consolidated semi-annual costs','Consolidated annual costs - FY','Consolidated annual costs - CY', 'clear outputs']
items2 = {}
for w2 in costs_types:
    if w2 == 'clear outputs':
        items2[w2] = widgets.Button(description=w2, layout=widgets.Layout(width='100%', height='40px'), button_style='warning', icon='check')
    else:
        items2[w2] = widgets.Button(description=w2,layout=widgets.Layout(width='100%', height='40px'))
        items2[w2].style.button_color = 'lightgreen'
left_box2 = widgets.VBox([list(items2.values())[0],list(items2.values())[1],list(items2.values())[2]],layout=box_layout)
right_box2 = widgets.VBox([list(items2.values())[3],list(items2.values())[4],list(items2.values())[5], list(items2.values())[6]],layout=box_layout)
items2['Semi-annual costs'].on_click(lambda x: display_with_out(costs.semi_costs,out2,input_dic))
items2['Annual costs - FY'].on_click(lambda x: display_with_out(costs.annual_costs,out2,input_dic))
items2['Annual costs - CY'].on_click(lambda x: display_with_out(costs.annual_costs,out2,input_dic,cal_year=True))
items2['Consolidated semi-annual costs'].on_click(lambda x: display_with_out(consolidated.costs_analysis,out2,input_dic))
items2['Consolidated annual costs - FY'].on_click(lambda x: display_with_out(consolidated.annual_costs_analysis,out2,input_dic))
items2['Consolidated annual costs - CY'].on_click(lambda x: display_with_out(consolidated.annual_costs_analysis,out2,input_dic,cal_year=True))
items2['clear outputs'].on_click(lambda x: display_with_out(clear_output,out2))
widgets.HBox([left_box2,right_box2])

In [ ]:
display(out2)

In [ ]:
ops = [i for i in list(range(general.recent_end_year, general.recent_end_year+10))]
a_slider = widgets.SelectMultiple(options=ops, description='Years for comparison')
display(a_slider)

compare1 = ['NNB comparison','AUA comparison']
box_layout2 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='30%')
items3 = {}
for w3 in compare1:
    items3[w3] = widgets.Button(description=w3,layout=widgets.Layout(width='100%', height='40px'))
    items3[w3].style.button_color = 'lightgreen'
box3 = widgets.Box([list(items3.values())[0],list(items3.values())[1]],layout=box_layout2)
items3['NNB comparison'].on_click(lambda x: display(consolidated.get_nnb_compare(data_dic, input_dic,list(a_slider.value))))
items3['AUA comparison'].on_click(lambda x: display(consolidated.get_aua_compare(data_dic, input_dic,list(a_slider.value))))


display(box3)

In [ ]:
compare2 = ['Interim revenue comparison','Annual revenue comparison', 'Interim costs comparison', 'Annual costs comparison']
box_layout3 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='50%')
items4 = {}
for w4 in compare2:
    items4[w4] = widgets.Button(description=w4,layout=widgets.Layout(width='100%', height='40px'))
    items4[w4].style.button_color = 'lightgreen'
top_box4 = widgets.HBox([list(items4.values())[0],list(items4.values())[1]],layout=box_layout3)
bottom_box4 = widgets.HBox([list(items4.values())[2],list(items4.values())[3]],layout=box_layout3)
items4['Interim revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,True,list(a_slider.value))))
items4['Annual revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,False,list(a_slider.value))))
items4['Interim costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,True,list(a_slider.value))))
items4['Annual costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,False,list(a_slider.value))))

widgets.VBox([top_box4, bottom_box4])

In [ ]:
'''=====current values=======
discounted cash flow period: 9 years
perpetuity growth rate: 0.035
cash flow discounted rate: 0.085
'''
out4 = Output()
dcf_period = widgets.IntText(value=9,placeholder='Integer',description='DCF period - No. of years:')
display(dcf_period)
perp_growth = widgets.FloatText(value=0.035,placeholder='Percentage as decimals',description='Perpetuity growth rate:')
display(perp_growth)
disct_rate = widgets.FloatText(value=0.085,placeholder='Percentage as decimals',description='Cash flow discount rate:')
display(disct_rate)
current_value = widgets.Checkbox(value=False, description='Fair value now')
display(current_value)
fractional = widgets.Checkbox(value=False, description='Fractional valuation')
display(fractional)

dis_cash = ['Discounted cash flow','Fair value', 'clear outputs']
items5 = {}
for w5 in dis_cash:
    if w5 == 'clear outputs':
        items5[w5] = widgets.Button(description=w5, layout=widgets.Layout(width='100%', height='40px'), button_style='warning', icon='check')
    else:
        items5[w5] = widgets.Button(description=w5, layout=widgets.Layout(width='100%', height='40px'))
        items5[w5].style.button_color = 'lightgreen'
fair = widgets.HBox([list(items5.values())[0],list(items5.values())[1],list(items5.values())[2]],layout=box_layout3)
items5['Discounted cash flow'].on_click(lambda x: display_with_out(discf.disc_cash_flow,out4,data_dic,input_dic,now=current_value.value,fractional=fractional.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value))
items5['Fair value'].on_click(lambda x: display_with_out(discf.fair_value,out4,data_dic,input_dic,now=current_value.value,fractional=fractional.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value,pep_rate=perp_growth.value))   
items5['clear outputs'].on_click(lambda x: display_with_out(clear_output,out4))
display(fair)

In [ ]:
display(out4)

In [ ]:
radio1 = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period')
display(radio1)

revenue_dist_radio = widgets.RadioButtons(options=[True, False],description='Semi-annual revenue distribution?')
display(revenue_dist_radio)


In [ ]:
out3 = Output()

others = ['HLF composite monthly return','Revenue distribution','Avg AUA distribution','Total AUA','Total NNB nnc Algo', 'NNB distribution nnc Algo', 'clear outputs']
items6 = {}

for w6 in others:
    if w6 == 'clear outputs':
        items6[w6] = widgets.Button(description=w6, layout=widgets.Layout(width='100%', height='40px'), button_style='warning', icon='check')
    else:
        items6[w6] = widgets.Button(description=w6, layout=widgets.Layout(width='100%', height='40px'))
        items6[w6].style.button_color = 'lightgreen'
other_box1 = widgets.VBox([list(items6.values())[0],list(items6.values())[1],list(items6.values())[2]],layout=box_layout)
other_box2 = widgets.VBox([list(items6.values())[3],list(items6.values())[4],list(items6.values())[5],list(items6.values())[6]],layout=box_layout)
items6['HLF composite monthly return'].on_click(lambda x: display_with_out(discretionary_aua.get_composite_return,out3,data_dic))
items6['Total AUA'].on_click(lambda x: display_with_out(combined.total_aua,out3,data_dic,input_dic))
items6['Revenue distribution'].on_click(lambda x: display_with_out(stats.summary_revenue_dist,out3,data_dic,input_dic,revenue_dist_radio.value))
items6['Avg AUA distribution'].on_click(lambda x: display_with_out(stats.summary_avg_aua_dist,out3,data_dic,input_dic,radio1.value))
items6['Total NNB nnc Algo'].on_click(lambda x: display_with_out(combined.total_nnb_ci_clientAlgo,out3,data_dic,input_dic, opt=radio1.value))
items6['NNB distribution nnc Algo'].on_click(lambda x: display_with_out(combined.total_nnb_distribution_clientAlgo,out3,data_dic, input_dic, opt=radio1.value))
items6['clear outputs'].on_click(lambda x: display_with_out(clear_output,out3))
widgets.HBox([other_box1, other_box2])

#Financial year based

In [ ]:
display(out3)

In [ ]:
out5 = Output()
nnb_radio = widgets.RadioButtons(options=['day','month','semi-annual','quarter','annual'],description='Period',disabled=False)
nnb_dropdown = widgets.Dropdown(options=['no_select',None],description='Fund option',disabled=False)
unit_dropdown = widgets.Dropdown(options=['acc','inc'],description='Unit type',disabled=False)
display(nnb_radio)
display(nnb_dropdown)
display(unit_dropdown)

nnb_b = widgets.Button(description='Implied HLF nnb period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b.style.button_color = 'lightgreen'

nnb_b2 = widgets.Button(description='HLF unit change period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b2.style.button_color = 'lightgreen'

fund_size_widget = widgets.Button(description='HLF total size', layout=widgets.Layout(width='50%', height='40px'))
fund_size_widget.style.button_color = 'lightgreen'

clear_outputs_widget = widgets.Button(description='clear outputs', layout=widgets.Layout(width='50%', height='40px'), button_style='warning', icon='check')

nnb_b.on_click(lambda x: display_with_out(stats.hlf_to_date_implied_nnb,out5,data_dic,typ=nnb_radio.value,fund_opt=nnb_dropdown.value))
nnb_b2.on_click(lambda x: display_with_out(stats.hlf_to_date_unit_change,out5,data_dic, unit_type=unit_dropdown.value,typ=nnb_radio.value,fund_opt=nnb_dropdown.value))
fund_size_widget.on_click(lambda x: display_with_out((lambda y=None: data_dic['fund size'].sum(axis='columns')),out5))
clear_outputs_widget.on_click(lambda x: display_with_out(clear_output,out5))

display(nnb_b)
display(nnb_b2)
display(fund_size_widget)
display(clear_outputs_widget)

In [ ]:
display(out5)

In [ ]:
out6 = Output()
cash_radio = widgets.RadioButtons(options=['monthly','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(cash_radio)


cash_b = widgets.Button(description='Cash margin', layout=widgets.Layout(width='50%', height='40px'))
cash_b.style.button_color = 'lightgreen'
clear_outputs_widget2 = widgets.Button(description='clear outputs', layout=widgets.Layout(width='50%', height='40px'), button_style='warning', icon='check')
cash_b.on_click(lambda x: display_with_out(stats.cash_margin,out6,data_dic, input_dic, cash_radio.value))
clear_outputs_widget2.on_click(lambda x: display_with_out(clear_output,out6))

display(cash_b)
display(clear_outputs_widget2)

In [ ]:
display(out6)

In [ ]:
out7 = Output()
eps_radio = widgets.RadioButtons(options=[True, False],description='Calendar year?',disabled=False)
display(eps_radio)

is_semi_radio = widgets.RadioButtons(options=[True, False],description='Semi-annual?',disabled=False)
display(is_semi_radio)

clear_outputs_widget3 = widgets.Button(description='clear outputs', layout=widgets.Layout(width='50%', height='40px'), button_style='warning', icon='check')

clear_outputs_widget3.on_click(lambda x: display_with_out(clear_output,out7))

eps_summary = widgets.Button(description='Net earnings and EPS', layout=widgets.Layout(width='50%', height='40px'))
eps_summary.style.button_color = 'lightgreen'
eps_summary.on_click(lambda x: display_with_out(stats.summary_total, out7, data_dic, input_dic, cal=eps_radio.value, is_semi=is_semi_radio.value))
display(eps_summary)
display(clear_outputs_widget3)


In [ ]:
display(out7)

In [ ]:
hlf_radio = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(hlf_radio)
out8 = Output()

buttom3 = ['HLF revenue margin','Monthly revenue', 'HLF avg fund size','Quarter Trading update revenue', 'clear outputs']
items7 = {}
for w7 in buttom3:
    if w7 == 'clear outputs':
        items7[w7] = widgets.Button(description=w7, layout=widgets.Layout(width='100%', height='40px'), button_style='warning', icon='check')
    else:
        items7[w7] = widgets.Button(description=w7, layout=widgets.Layout(width='100%', height='40px'))
        items7[w7].style.button_color = 'lightgreen'

top_m = widgets.HBox([list(items7.values())[0],list(items7.values())[1]],layout=box_layout3)
bottom_m = widgets.HBox([list(items7.values())[2],list(items7.values())[3],list(items7.values())[4]],layout=box_layout3)

items7['HLF revenue margin'].on_click(lambda x: display_with_out(stats.hlf_revenue_margin,out8,data_dic,input_dic,hlf_radio.value))
items7['Monthly revenue'].on_click(lambda x: display_with_out(revenue.monthly_revenue,out8,data_dic, input_dic))
items7['HLF avg fund size'].on_click(lambda x: display_with_out(stats.avg_hlf_size,out8,data_dic, input_dic,hlf_radio.value))
items7['Quarter Trading update revenue'].on_click(lambda x: display_with_out(stats.quarter_revenue,out8,data_dic, input_dic))
items7['clear outputs'].on_click(lambda x: display_with_out(clear_output,out8))


widgets.VBox([top_m, bottom_m])

In [ ]:
display(out8)

In [ ]:
display(a_slider)

out9 = Output()

client_options = ['Total No. of clients','Net new clients','fx effects on funds','clear outputs']
items8 = {}
for w8 in client_options:
    if w8 == 'clear outputs':
        items8[w8] = widgets.Button(description=w8, layout=widgets.Layout(width='100%', height='40px'), button_style='warning', icon='check')
    else:
        items8[w8] = widgets.Button(description=w8, layout=widgets.Layout(width='100%', height='40px'))
        items8[w8].style.button_color = 'lightgreen'
c_fair = widgets.HBox([list(items8.values())[0],list(items8.values())[1],list(items8.values())[2],list(items8.values())[3]],layout=box_layout3)
items8['Total No. of clients'].on_click(lambda x: display_with_out(combined.total_client_predt,out9,data_dic, input_dic, slice=list(a_slider.value)))
items8['Net new clients'].on_click(lambda x: display_with_out(combined.net_new_client_predt,out9,data_dic, input_dic, slice=list(a_slider.value)))
items8['fx effects on funds'].on_click(lambda x: display_with_out(vantage_aua.fx_effects_on_funds,out9,data_dic))
items8['clear outputs'].on_click(lambda x: display_with_out(clear_output,out9))

display(c_fair)

In [ ]:
display(out9)

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:

# # =============================== fair value sensitivity analysis ===========================================

# disc_rate_list = [0.0825,0.085,0.0875,0.09,0.095,0.1]
# pep_rate_list = [0.035,0.0375,0.04,0.0425,0.045]

# df_1y = pandas.DataFrame(columns = disc_rate_list, index=pep_rate_list)
# df_1y.columns.name = 'Cost of Equity'
# df_1y.index.name = 'Long-term Growth Rate'

# for i in pep_rate_list:
#     for j in disc_rate_list:
#         df_1y.loc[i,j] = discf.fair_value(data_dic, input_dic,now=False,fractional=True,dcf_p=9,disc_rate=j,pep_rate=i).loc['Fair value per share','HL']


In [ ]:
# df_1y